In [31]:
import zarr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import csv

In [2]:
def load_single_data(i):
    data = zarr.open('H:\Desktop\Q1\python\project\TIL-Group15\dataset\CarFollowing/trainHH.zarr', mode='a')
    start, end = data.index_range[i]
    # get timestamps
    timestamps = data.timestamp[start:end]
    # get position, speed, and acceleration
    
    x_lead = data.lead_centroid[start:end]
    v_lead = data.lead_velocity[start:end]
    a_lead = data.lead_acceleration[start:end]
    id=np.ones((1,x_lead.shape[0]))*i
    x_follow = data.follow_centroid[start:end]
    v_follow = data.follow_velocity[start:end]
    a_follow = data.follow_acceleration[start:end]
    array = np.vstack((id, x_lead, v_lead, a_lead, x_follow, v_follow, a_follow, timestamps))
    return array.T

def load_size(i):
    data = zarr.open('H:\Desktop\Q1\python\project\TIL-Group15\dataset\CarFollowing/trainHH.zarr', mode='a')
    #size_lead = 4.85              # this is for AV
    size_lead = data.lead_size[i] # this is for HV
    size_follow = data.follow_size[i]
    id=np.ones(1)*i
    array = np.vstack((id, size_lead, size_follow)) 
    return array.T

def load_multi_size(i):
    multi_data = None
    for x in range(i + 1):
        single_data1 = load_size(x)
        if x == 0:
            multi_data = single_data1
        else:
            multi_data = np.vstack([multi_data, single_data1])
    return multi_data
def load_multi_data(i):
    multi_data = None
    for x in range(i+1):
        single_data = load_single_data(x)
        if x == 0:
            multi_data = single_data
        else:
            multi_data = np.vstack([multi_data, single_data])
    return multi_data
p=int(input('type in the number you want'))
np.savetxt( "multi_data.csv", load_multi_data(p), delimiter="," )
np.savetxt( "multi_size.csv", load_multi_size(p), delimiter="," )

In [3]:
data=np.loadtxt('multi_data.csv',delimiter="," )
data_size=np.loadtxt('multi_size.csv',delimiter="," )
#data_size.columns = ["ID", "size_lead", "size_follow"]
#data.columns = ["ID","x_lead", "v_lead", "a_lead", "x_follow", "v_follow", "a_follow", "timestamps"]


In [4]:
for i in range(p):
    x_data=np.squeeze(data[np.where(data[:,0]==i),:],0)
    x_size=np.squeeze(data_size[np.where(data_size[:,0]==i),:],0)
    
    x_lead=x_data[:,1]
    x_follow=x_data[:,4]
    size_lead = x_size[:,1]
    size_follow = x_size[:,2]
    A_distance=[]
    B_distance=[]
    for y in range(1,x_lead.size):
        A = x_lead[y] - x_follow[y] - 0.5 * (size_lead[0] + size_follow[0])
        B = x_lead[y] - x_follow[y] - 0.5 * (size_follow[0] - size_lead[0])
        A_distance.append(A)
        B_distance.append(B)
    array = np.vstack((A_distance,B_distance))
    np.savetxt( "size.csv", array.T, delimiter="," ) 

In [18]:

data3 = np.genfromtxt('regimes_list_HH_train.csv', delimiter=',',  dtype=None, names=True)

# 定义查找模块的顺序
pattern = re.compile(r'Fa|Fd|A|D|F|C|S')

output_list = []  # 用于存储结果的列表

# 迭代每行数据
for row in data:
    regime = row['regime_comb'].decode('utf-8')  # 如果文件是utf-8编码，可能需要解码
    matches = pattern.findall(regime)  # 使用正则表达式来分割regime_comb值
    row_dict = {}  # 用于当前行的模块和时间的字典
    for module in matches:
        row_dict[module] = row[module]
    
    # 将当前行的字典添加到结果列表中
    output_list.append(row_dict)

# 打印结果
for item in output_list:
    print(item)

C:\Users\1\AppData\Local\Temp\ipykernel_19528\1242068403.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  data3 = np.genfromtxt('regimes_list_HH_train.csv', delimiter=',',  dtype=None, names=True)


{'Fa': 0.6000000000000001, 'A': 7.1000000000000005, 'D': 0.2, 'F': 10.0}
{'Fa': 0.7, 'C': 1.1, 'A': 7.8, 'F': 12.3}
{'Fa': 1.1, 'A': 7.8, 'D': 0.7, 'F': 12.3}
{'A': 2.3000000000000003, 'F': 19.6}
{'Fa': 1.2, 'C': 6.0, 'A': 0.6, 'D': 0.4, 'F': 13.700000000000001}
{'C': 7.300000000000001, 'A': 1.2, 'D': 1.1, 'F': 11.3}
{'C': 9.4, 'D': 2.4, 'F': 9.7, 'Fd': 0.4}
{'D': 1.3, 'F': 20.6}
{'C': 0.5, 'A': 3.0, 'D': 4.8, 'F': 13.6}
{'A': 1.0, 'D': 1.9, 'F': 13.0}
{'Fa': 8.5, 'C': 1.5, 'F': 5.9}
{'Fa': 9.0, 'C': 8.1, 'F': 6.8}
{'Fa': 7.1000000000000005, 'A': 6.6, 'F': 4.2}
{'Fa': 4.2, 'C': 1.7, 'A': 4.3, 'D': 2.0, 'F': 7.7, 'Fd': 2.0}
{'A': 3.8000000000000003, 'D': 1.8, 'F': 16.3}
{'Fa': 2.3000000000000003, 'C': 13.4, 'A': 2.9, 'D': 2.7, 'F': 1.0999999999999999, 'S': 1.5}
{'C': 1.0999999999999999, 'A': 3.3, 'D': 0.30000000000000004, 'F': 11.2}
{'A': 1.4000000000000001, 'D': 0.4, 'F': 16.1}
{'A': 2.6, 'F': 13.3}
{'A': 0.1, 'D': 0.30000000000000004, 'F': 15.5}
{'C': 4.199999999999999, 'A': 0.6000000

In [22]:
for i in range(len(output_list)):
    pattern_times =output_list[i]

# 速度数据
    speeds = data[data[:,0]==i]  # 你提供的速度数据列表

# 为每个模式切割速度数据
pattern_speeds = {}
start_index = 0
for pattern, time in pattern_times.items():
    # 计算该模式对应的速度数据的长度
    length = int(time * len(speeds) / sum(pattern_times.values()))
    pattern_speeds[pattern] = speeds[start_index:start_index+length]
    start_index += length

# 打印结果
for pattern, speed_data in pattern_speeds.items():
    print(pattern, speed_data)

<function <lambda> at 0x00000200B38E9D00>


In [33]:
data = np.loadtxt('multi_data.csv', delimiter=",")
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # 写入标题/表头
    writer.writerow(['ID', 'Pattern', 'Speed'])

    for i in range(len(output_list)):
        pattern_times = output_list[i]
        speeds = data[data[:, 0] == i, 1]
        pattern_speeds = {}
        start_index = 0
        for pattern, time in pattern_times.items():
            length = int(time * len(speeds) / sum(pattern_times.values()))
            pattern_speeds[pattern] = speeds[start_index:start_index+length]
            start_index += length

        # 将结果保存到CSV中
        for pattern, speed_data in pattern_speeds.items():
            for speed in speed_data:
                writer.writerow([i, pattern, speed])